In [1]:
import sys
from pathlib import Path

sys.path.append(str((Path("..").resolve().absolute())))

from SPC_2L.DataTools import DataBase, to_img, show
import pickle
from SPC_2L.Network import LayerPC, Network
from SPC_2L.Coding import ML_FISTA, ML_Lasso
import torch.nn.functional as f
import matplotlib.pyplot as plt
import numpy as np
from torchvision.utils import make_grid, save_image
import torch
from torchvision.transforms import ToTensor,Compose
from SPC_2L.DataTools import LCN, whitening, z_score, mask, to_cuda, norm
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

In [2]:
cmap1=plt.get_cmap('tab20c')
cmap2=plt.get_cmap('tab20b')
color_fb=[cmap1(21),cmap1(3),cmap1(2),cmap1(1),cmap1(0)]

In [3]:
data_path = '../DataSet/MNIST/'

transform = Compose([ToTensor(),
                    to_cuda(),
                    whitening((28,28),f_0=0.6),
                    LCN(kernel_size=3,sigma=0.5,rgb=False),
                    z_score()])

dataset = MNIST(data_path, transform=transform, train=False, download=True)

DataBase =  DataLoader(dataset,batch_size=32,shuffle=True,drop_last=True)

nb_image = 320

In [4]:
feedback_list = [0,1,2,3,4]
where = 'E:\\SDPC\\SDPC\\notebooks\\Savings\\MNIST\\'
model_name = 'MNIST_[0.2,0.3]_b=0.pkl'
saving_path = where + 'Saving_MNIST_ActivityPercent.pkl'

model_name_list = ["E:\\SDPC\\SDPC\\notebooks\\Savings\\MNIST\\MNIST_[0.2,0.3]_b=0.pkl"]

all_sparsity = torch.zeros(nb_image,len(feedback_list),2).cuda()
all_activity = torch.zeros(len(DataBase.dataset), len(feedback_list),2).cuda()

for k_FB in feedback_list:
    idx_model=k_FB
    k=0
    model_type = model_name[0:8]
    name = model_name

    path = where + name
    with open(path, 'rb') as file:
        out = pickle.load(file)
    Net = out['Net']
        
    for i in range(Net.nb_layers):
        Net.layers[i].b = k_FB
    Loss = ML_Lasso(Net, [0.2,0.3])
    Pursuit = ML_FISTA(Net, Loss, max_iter=1000, th=1e-4, mode='eigen', accell=True)
    for idx_batch, data in enumerate(DataBase):
        batch = data[0].cuda()
        gamma, it, Loss_G, delta = Pursuit.coding(batch)
        for i in range(Net.nb_layers):
            nb_null = (gamma[i]).float().view(gamma[i].size(0),-1)
            min = nb_null.max() - nb_null.min()
            nb_null -= nb_null.min()
            nb_null /= min
            volume = gamma[i].size(1)*gamma[i].size(2)*gamma[i].size(3)
            all_activity[k:k+batch.size(0),k_FB,i] += nb_null.sum(-1)/volume*100
        k+=batch.size(0)
        if k >= nb_image:
            break

Exp = {'model_name': model_name_list,
        'activity' : all_activity}
print(all_activity.mean(dim=0))
print(all_activity.std(dim=0))

tensor([[0.0241, 0.0211],
        [0.0257, 0.0231],
        [0.0255, 0.0301],
        [0.0630, 0.1033],
        [   nan,    nan]], device='cuda:0')
tensor([[0.1339, 0.1185],
        [0.1426, 0.1280],
        [0.1414, 0.1666],
        [0.5106, 0.7652],
        [   nan,    nan]], device='cuda:0')


In [9]:
save_fig = False
figsize = (6.4,5)

labels = ['$k_{FB}=0$','$k_{FB}=1$','$k_{FB}=2$','$k_{FB}=3$','$k_{FB}=4$']

mean_sp = Exp['activity'].transpose(0,1)[0].mean(dim=0)
std_sp = Exp['activity'].transpose(0,1)[0].std(dim=0)
model_name_list = Exp['model_name']
list_layer= np.arange(len(model_name_list))
N = len(list_layer)
position = [1 + 1*i for i in range(N)] 


width = 0.75 

fig = plt.figure(figsize=figsize)

plot_SDPC = plt.bar(position, 100-mean_sp[:,0].numpy(),  yerr=std_sp[:,0].numpy(), align='center',
                 capsize=2,width=0.75, color=['slategrey',cmap1(3),cmap1(2),cmap1(1),cmap1(0)])

plt.ylabel('% of active neurons', fontsize=15, labelpad=10)
xticks_position = [1 +1*i for i in range(N)]
plt.xlabel('feedback strength '+ r'($k_{FB}$)', fontsize=15, labelpad=10)
plt.xticks(xticks_position,list_layer , fontsize=15)
#plt.title('a) - First Layer sparsity when increasing the \nfeedback strenght', fontsize=15, pad=15)
#plt.ylim()
plt.yticks([0,1,2,3,4,5, 6, 7, 8, 9, 10], fontsize=15)
#plt.title('i) - AT&T', fontsize=15)
'''
for i in range(5):
    plot_SDPC[i].set_label(labels[i])


plt.legend(loc='lower center',bbox_to_anchor=(0.5, -0.5),ncol=5,fontsize=15)
'''
plt.tight_layout()

if save_fig == True:
    '''
    plt.savefig('Fig3/histo_legend.png', bbox_inches = 'tight', pad_inches = 0.5, dpi= 400)
    '''
    plt.savefig('Savings/Fig/Fig3/Fig3a.png', bbox_inches = 'tight', pad_inches = 0, dpi= 400)
    

plt.show()


IndexError: too many indices for tensor of dimension 0

<Figure size 640x500 with 0 Axes>